In [ ]:
from plantcv import plantcv as pcv
import videometer.hips as hips
import numpy as np
import argparse
import glob
from random import shuffle
import plantcv.plantcv as pcv
from skimage import img_as_ubyte

In [ ]:
# Set global debug behavior to None (default), "print" (to file), 
# or "plot" (Jupyter Notebooks or X11)
pcv.params.debug = "plot"
pcv.params.text_size = 5
pcv.params.text_thickness = 5

In [ ]:
def main():
    indir = r'F:\Landrigan\envi'
    outdir = r'F:\Landrigan\KernelWingFunctionReview'
    plot = "TRUE"
    
    files = glob.glob(f"{indir}/*.raw")
    shuffle(files)

    for f in files:
        print(f)
        ## read image, create full seed mask
        img = pcv.readimage(filename=f, mode="envi")
        c_img = pcv.rgb2gray_hsv(rgb_img=img.pseudo_rgb, channel='h')
        c_thresh = pcv.threshold.mean(gray_img=c_img, ksize=22000, offset=40, object_type='dark')
        c_fill = pcv.fill_holes(bin_img=c_thresh)
        roi = pcv.roi.rectangle(img=img.pseudo_rgb, x=277, y=100, h=2700, w=3700)
        a_fill_image = pcv.fill(bin_img=c_fill, size=18000)
        flt_mask = pcv.roi.quick_filter(mask=a_fill_image, roi=roi)
        try:
            rois = pcv.roi.auto_grid(mask=flt_mask, nrows=4, ncols=6, radius=250, img=img.pseudo_rgb)
        except RuntimeError as err:
            print(f"{f} failed at auto_grid(); skipping")
            continue
        except ValueError as err:
            print(f"{f} failed at auto_grid(); skipping")
            continue
        labeled_mask, num = pcv.create_labels(mask=flt_mask, rois=rois)
    
        # make wing mask
        r940 = img_as_ubyte(img.array_data[: , :, -2] / 255)
        masked_r940 = pcv.apply_mask(img=r940, mask=flt_mask, mask_color="black")
        wing_mask = pcv.threshold.binary(gray_img=masked_r940, threshold=40, object_type="light")
        wing_err = pcv.erode(gray_img=wing_mask, ksize=4, i=1)
        wing_mask_dil = pcv.dilate(gray_img=wing_err, ksize=3, i=1)
        wing_mask_clean = pcv.fill(bin_img=wing_mask_dil, size=2500)
        wing_labeled_mask, num = pcv.create_labels(mask=wing_mask_clean, rois=rois)
    
        # make kernel mask
        ## subtract wing mask from full mask to leave kernels
        kernel_mask = flt_mask - wing_mask
        ## errode kernel mask to remove remaining white space in mask
        kernel_err = pcv.erode(gray_img=kernel_mask, ksize=4, i=5)
        kernel_mask_clean = pcv.fill(bin_img=kernel_err, size=2900)
        kernel_mask_dil = pcv.dilate(gray_img=kernel_mask_clean, ksize=4, i=4)
        kernel_labeled_mask, num = pcv.create_labels(mask=kernel_mask_dil, rois=rois)
        
        ##extract shape and spectral data per seed
        #whole seed data extraction
        seed_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=labeled_mask, n_labels=num, label="seed")
        seed_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=labeled_mask, n_labels=num, label="seed")
        seed_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=labeled_mask, n_labels=num, label='seed')
    
        #wing data extraction
        wing_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=wing_labeled_mask, n_labels=num, label="wing")
        wing_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=wing_labeled_mask, n_labels=num, label="wing")
        wing_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=wing_labeled_mask, n_labels=num, label='wing')
    
        #kernel data extraction
        kernel_shape_img = pcv.analyze.size(img=img.pseudo_rgb, labeled_mask=kernel_labeled_mask, n_labels=num, label="kernel")
        kernel_hsv_img = pcv.analyze.color(rgb_img=img.pseudo_rgb, labeled_mask=kernel_labeled_mask, n_labels=num, label="kernel")
        kernel_analysis_img = pcv.analyze.spectral_reflectance(hsi=img, labeled_mask=kernel_labeled_mask, n_labels=num, label='kernel')
    
        if plot == "TRUE":
            outfile = f.removesuffix('.raw').removeprefix(indir)
            #save seed mask for each plate
            x = pcv.visualize.pseudocolor(c_fill, mask=labeled_mask, background='black')
            x.savefig(f'{outdir}/{outfile} Seed.png')
            #save wing mask for each plate
            y = pcv.visualize.pseudocolor(c_fill, mask=wing_labeled_mask, background='black')
            y.savefig(f'{outdir}/{outfile} Wing.png')
            #save kernel mask for each plate
            z = pcv.visualize.pseudocolor(c_fill, mask=kernel_labeled_mask, background='black')
            z.savefig(f'{outdir}/{outfile} Kernel.png')
    
        #save outputs
        print(outdir)
        outfile = f.removesuffix('.raw').removeprefix(indir)
        pcv.outputs.save_results(filename=f"{outdir}/{outfile}.csv", outformat="csv")

    return 1
if __name__ == '__main__':
    main()